In [ ]:
%gui qt6
# 주피터 환경에서 PyQt6 를 띄우기 위한 명령어
# 라이브러리
import sys   # 파이썬 시스템 조작 도구
import random  # 진동 효과를 위한 무작위 숫자 뽑기 위해
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QFrame, QGraphicsOpacityEffect, QVBoxLayout, QPushButton  # 화면 구성 위젯( 버튼, 라벨 ,로봇 입 등)
from PyQt6.QtCore import Qt, QPropertyAnimation, QRect, QTimer, QParallelAnimationGroup # 시간,위치(QRect), 정렬(QT)등 핵심기술

# PyQt 앱을 실행하기 위한 QApplication 객체 생성 쥬피터 노트에서는 반드시 필요!
app = QApplication.instance()
if not app:
    app = QApplication(sys.argv)   # 이미 존재하면 재사용, 없으면 새로 만듦

# 로봇 얼굴 화면 (메인 1 화면)
class RobotMainScreen(QWidget):  #QWidget 상속받은 메인화면
    def __init__(self, parent=None):  # 생성자
        super().__init__(parent)  # 부모클래스 기능 그대로
        self.setFixedSize(800, 600)  # 창 크기를 800x600으로 고정
        self.setStyleSheet("background-color: #050505;")  # 창 전체 배경색을 칠해 빈틈 방지 다크 그레이색

        # 로봇의 귀여운 뿔 (안테나)
        self.ant_l_stick = QFrame(self)  # 직선형 기둥
        self.ant_l_stick.setGeometry(270, 50, 15, 60)
                     # 위젯의 위치,크기(가로,세로,너비,높이)
        self.ant_l_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
                    # 위젯의 모든면에서 3픽셀 두께의 흰색 선을 그려라!, 테두리 중 아래만 없애라! , 배경은 투명
        self.ant_l_tip = QFrame(self)
        self.ant_l_tip.setGeometry(252, 35, 50, 25)
        self.ant_l_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #FF8033;")  # 주황색

        self.ant_r_stick = QFrame(self)
        self.ant_r_stick.setGeometry(515, 50, 15, 60)
        self.ant_r_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        self.ant_r_tip = QFrame(self)
        self.ant_r_tip.setGeometry(497, 35, 50, 25)
        self.ant_r_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #00A3E8;") # 하늘색, 바다색

        # 로봇 얼굴 : face_outline
        self.face_outline = QFrame(self)
        self.face_outline.setGeometry(150, 110, 500, 330)
        self.face_outline.setStyleSheet("border: 4px solid #4364F7; border-radius: 60px; background-color: #111;")
                                                        # 로열 블루 ,모서리 끝점에서 가로 세로 60px지점을 부드러운 원형 곡선으로 , #020202 보다는 살짝 더 밝은 검은색

        # 눈 버튼
        self.btn_manual = QPushButton("직접 이동", self.face_outline)
        self.btn_manual.setGeometry(60, 70, 160, 110)  # #1A1A1A 다크 그레이, #FF8033주황색, solid 는 실선, #FF8033주황색
        self.btn_manual.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #FF8033;
                font-size: 18px; font-weight: bold; border: 2px solid #FF8033; border-radius: 25px;
            }
            QPushButton:hover { background-color: #FF8033; color: black; }
        """)  # 마우스를 올릴때만 작동

        self.btn_auto = QPushButton("자동 이동", self.face_outline)
        self.btn_auto.setGeometry(280, 70, 160, 110)   # #00A3E8 맑고 깊은 파란색
        self.btn_auto.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #00A3E8;
                font-size: 18px; font-weight: bold; border: 2px solid #00A3E8; border-radius: 25px;
            }
            QPushButton:hover { background-color: #00A3E8; color: black; }
        """)

        # '=' 입 모양
        self.mouth_top = QFrame(self.face_outline)
        self.mouth_top.setGeometry(210, 240, 80, 5)
        self.mouth_top.setStyleSheet("background-color: #00FF00; border-radius: 2px;")  # #00FF00 형광연두색
        self.mouth_bottom = QFrame(self.face_outline)
        self.mouth_bottom.setGeometry(210, 255, 80, 5)
        self.mouth_bottom.setStyleSheet("background-color: #00FF00; border-radius: 2px;")

        # 하단 안내 칸
        self.bottom_bar = QFrame(self)
        self.bottom_bar.setGeometry(150, 500, 500, 70)
        self.bottom_bar.setStyleSheet("border: 2px solid white; background-color: #222; border-radius: 20px;") #222 목탄색
        self.bottom_label = QLabel("이동 방식을 선택해주세요.", self.bottom_bar)
        self.bottom_label.setGeometry(0, 0, 500, 70)
        self.bottom_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 글자를 정중앙으로!!
        self.bottom_label.setStyleSheet("color: white; font-size: 18px; font-weight: bold; border: none;")

# 대장 클래스 (전체 배경색 적용)
class LegosApp(QWidget): # 전체 GUI 담당
    def __init__(self):
        super().__init__()
        self.setFixedSize(800, 600) # 가로,세로 테두리 없는 창
        self.setWindowFlags(Qt.WindowType.FramelessWindowHint) # 위쪽의 닫기/최소화바 없앰.
        self.setStyleSheet("background-color: #050505;")  # 전체 앱 배경을 검은색으로 고정하여 테두리 빈틈 방지
        # 하단 레이어 : 메인1 화면
        self.main_view = RobotMainScreen(self)
        self.main_view.setGeometry(0, 0, 800, 600)
        # 상단 레이어: 인트로
        self.intro_layer = QFrame(self)
        self.intro_layer.setGeometry(0, 0, 800, 600)
        self.intro_layer.setStyleSheet("background-color: #050505;")  # 완전 검은색

        # 로고 설정 (인트로 레이어 소속)
        self.logo = QLabel("LEGOS", self.intro_layer)
        self.logo.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 정중앙
        self.logo.setGeometry(0, 225, 800, 150)  # 화면 중앙 높이쯤 배치
        self.current_spacing = -60   # 처음에 글자 뭉치게 자간 -60
        self.update_logo_style()  # 아래의 정의된 스타일 함수 호출
        # 로고가 '스르륵' 나타나게 해주는 투명도 필름(Effect) 장착
        self.logo_op = QGraphicsOpacityEffect(self.logo)
        self.logo.setGraphicsEffect(self.logo_op)
        self.logo_op.setOpacity(0) # 시작은 투명 0 (안보임)
        # 문 2개 설정 (인트로 레이어 소속)
        self.left_door = QFrame(self.intro_layer)
        self.right_door = QFrame(self.intro_layer)
        self.setup_full_design_doors()  # 디자인 (못,손잡이, 경고띠) 입힘
        # 문 흔들기 애니메이션 시작 진동을 제어할 변수와 타이머 준비
        self.shake_count = 0  # 몇번 진동 했는지 기록
        self.shake_timer = QTimer(self)  # 째깍거리는 시계
        self.shake_timer.timeout.connect(self.do_shake)  # 시계가 울릴때마다 do_shake 실행 무작위로 x,y 이동! 진동 효과
        QTimer.singleShot(1000, self.start_shaking) # 1초 후에 start_shaking 실행

    def update_logo_style(self):  # 로고 스타일
                               #  글자 색상 : 선형그래디언트   시작점(0,0)위, 끝점(0,1)아래,위쪽끝:연한파랑,중간색 : 진한 파랑, 아래끝:짙은 파랑
        self.logo.setStyleSheet(f"color: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #6FB1FC, stop:0.5 #4364F7, stop:1 #0052D4); font-size: 150px; font-weight: bold; letter-spacing: {self.current_spacing}px; background: transparent;")
         #크기   :   150   ,             굵게 ,   글자 간격       -60~15  널게 벌어짐       , 글자배경 :  투명 배경없음,배경색 그대로
    def setup_full_design_doors(self):  # 문 디테일
        # 오리지널 CSS 스타일 그대로 적용 CSS 코드들 변수에 저장(가독성위해)
        door_base_css = "background: #111; border: 1px solid #222;"   # 문 바디
                        # 배경 : 진한 검정색  , 테두리 1픽셀 두께 , 어두운 회색
        #패널 문안쪽 장식판 # 배경                   왼쪽   위   , 오른쪽 아래   대각선 방향으로 밝은회색->짙은회색     테두리 : 2픽셀, 회색
        panel_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #2a2a2a, stop:1 #151515); border: 2px solid #3d3d3d; border-radius: 4px;"    # 모서리 살짝 둥글게
        # 못(리벳)    # 배경       원형그라디언트       중심         , 반지름       중심색:연회색, 바깥색 : 검정  , 동그란 못 모양
        rivet_css = "background: qradialgradient(cx:0.4, cy:0.4, radius:0.7, stop:0 #ccc, stop:1 #111); border-radius: 7px;"
        # 손잡이  (메탈 느낌)                        왼쪽       , 오른쪽      , 왼쪽끝  검정   중앙     흰색   오른쪽끝 검정  ,  어두운 테두리
        handle_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:0, stop:0 #000, stop:0.5 #fff, stop:1 #000); border: 1px solid #444;"
        # 경고띠                      선형 그라디언트     반복       ,                                     노랑
        hazard_css = "background: qlineargradient(spread:reflect, x1:0, y1:0, x2:0.4, y2:0.1, stop:0 #fbc02d, stop:0.5 #fbc02d, stop:0.51 #000, stop:1 #000);"  # 검정 교차

        for door, side in [(self.left_door, "L"), (self.right_door, "R")]:
            # 위젯 위치와 크기 지정                          너비,높이 (문크기)
            door.setGeometry(0 if side=="L" else 400, 0, 400, 600)
                            # 왼쪽(0,0,400,600) , 오른쪽 (400,0,400,600)
            door.setStyleSheet(door_base_css) # "background: #111; border: 1px solid #222;"검정, 1픽셀의 짙은회색

            # 1. 직사각형 패널
            panel = QFrame(door)  # 사각형 빈틀
            panel.setGeometry(80, 100, 240, 400)
            # 위치 왼쪽,오른쪽 숫자만큼 떨어진,크기
            panel.setStyleSheet(panel_css)

            # 2. 못 (리벳)
            pts = [(15, 15), (210, 15), (15, 370), (210, 370)]  # 못 위치
            for px, py in pts:
                rv = QFrame(panel)
                rv.setGeometry(px, py, 14, 14)  # 크기
                rv.setStyleSheet(rivet_css)

            # 3. 메탈 손잡이
            h = QFrame(door)
            h.setGeometry(385 if side=="L" else 0, 250, 15, 100)
               # 위치  :문 오른쪽 끝       ,  문 왼쪽 끝 근처
            h.setStyleSheet(handle_css)

            # 4. 공장 무늬 (경고띠)
            haz = QFrame(door)
            haz.setGeometry(0, 560, 400, 40)
                   # 위치  문 하단  , 너비 , 높이
            haz.setStyleSheet(hazard_css)

    def start_shaking(self): self.shake_timer.start(15)  # 0.015 초마다 진동 알람.
    def do_shake(self):  # 알람이 울릴 때마다 실행되는 함수
        power = 10 + (self.shake_count // 20)
        # 강도 기본 10  + 20번 마다 1px 증가
        dx, dy = random.randint(-power, power), random.randint(-power, power) # 랜덤 가로, 세로 값
        self.left_door.move(dx, dy); self.right_door.move(400 + dx, dy)  # 문을 랜덤위치로~
        self.shake_count += 1
        if self.shake_count > 80:  # 80번까지 진동!
            self.shake_timer.stop() # 알람 stop
            self.left_door.move(0, 0); self.right_door.move(400, 0)  # 문 위치 다시 0으로 정렬
            self.start_opening() # 문열기

    def start_opening(self):
        self.group = QParallelAnimationGroup()  # 동시에 여러 애니메이션 돌리기 위한 grop
        # 2.4초 동안 문을 화면 밖으로 완전히 밀어냄
        # QPropertyAnimation PyQt에서 위젯 속성을 시간에 따라 애니메이션 하는 클래스
        # PyQt의 QPropertyAnimation은 속성 이름을 문자열이 아닌 바이트 문자열로 받도록 되어 있음 그래서 b. 이렇게 씀
        # geometry = QRect(x, y, width, height)
        a1 = QPropertyAnimation(self.left_door, b"geometry"); a1.setDuration(2400); a1.setEndValue(QRect(-400, 0, 400, 600))
                         # 애니메이션 적용 위젯, 속성(위치, 크기)           2.4초 움직임  ,       완쪽 화면 밖 , 문크기 유지
        a2 = QPropertyAnimation(self.right_door, b"geometry"); a2.setDuration(2400); a2.setEndValue(QRect(800, 0, 400, 600))
        a3 = QPropertyAnimation(self.logo_op, b"opacity"); a3.setDuration(2500); a3.setEndValue(1)
                       # QLabel로고에 적용한 투명도 효과,속성                    투명도 1 완전히 보임 문이 열리는 즉시 로고 보임.

        self.group.addAnimation(a1); self.group.addAnimation(a2); self.group.addAnimation(a3)
        self.group.start()  # 3개의 애니메이션 동시 시작!
        # 글자 자간을 벌리는 타이머도 별도로 작동
        self.st = QTimer(self); self.st.timeout.connect(self.anim_text); self.st.start(30)

    def anim_text(self):
        if self.current_spacing < 15:  # 자간이 15 될 때까지
            self.current_spacing += 1.0; self.update_logo_style()   # 1픽셀씩 벌리고 로고 스타일 업데이트
        else:
            self.st.stop()  # 다 벌어지면 stop
            # 연출 감상을 위해 1.8초 대기 후 인트로 종료 함수 실행
            QTimer.singleShot(1800, self.finish_intro)

    def finish_intro(self):
        # 실행 되는 순간 인트로 레이어를 삭제하며 밑에 있던 메인 화면이 매끄럽게 보임
        self.intro_layer.deleteLater()

    def keyPressEvent(self, event):
        if event.key() == Qt.Key.Key_Escape: self.close()   # esc 누르면 꺼짐.

# 실행
main_window = LegosApp()  # 생성자 실행 되며 세팅
main_window.show()  # 보여줘!